# Installation

In [1]:
#!pip install z3-solver

In [2]:
from z3 import *
set_option(html_mode=False)

## Sorts

Sorts are the datatypes that the SMT solver can work on.

In [3]:
# Bool
x = Bool('a')

a = BoolVal(True)
b = BoolVal(False)

# Integers
x = Int('x')
y = Int('y')

a = IntVal(5)

# Real numbers
x = Real('x')
y,z = Reals("y z")

a = RealVal(3.141)
b = Q(1,3) # 1/3

# Bit-vector
x = BitVec('x', 16) # 16 Bit
y = BitVec('y', 32) # 32 Bit

a = BitVecVal(16, 32) # 32 Bit vector with value 16 (#x00000010)


In [4]:
# Common Mistake with rational numbers
print(RealVal(1/3))
print(Q(1,3))

# Gets internal, symbolic-representation in Lisp-like notation
print(RealVal(1/3).sexpr())
print(Q(1,3).sexpr())

3333333333333333/10000000000000000
1/3
(/ 3333333333333333.0 10000000000000000.0)
(/ 1.0 3.0)


### Preview: Composite Sorts
- Arrays
- Tuples
- Records
- Enumerations

and User Defined Sorts.

## Using the Z3-Solver

### System of linear equations
$$
\begin{align}
3x + 2y -z &= 1 \\
2x-2y+4z &= -2 \\
-x +  \frac{1}{2}y -z &=0
\end{align}
$$

In [5]:
x,y,z = Reals("x y z")

s = Solver()
s.add(3*x + 2*y -z == 1)
s.add(2*x - 2*y +4*z == -2)
s.add(-1*x + Q(1,2) * y - z == 0)

if s.check() == sat:
    m = s.model()
    x = m.evaluate(x)
    y = m.evaluate(y)
    z = m.evaluate(z)
    print(f"{x=} {y=} {z=}")
else:
    print(s.check())

x=1 y=-2 z=-2


$$
\begin{align}
2x - 4y +z &= 3 \\
8x-2y+4z &= 7 \\
-4x +  y -2z &=14
\end{align}
$$

In [6]:
x,y,z = Reals("x y z")

s = Solver()
s.add(2*x - 4*y + z == 3)
s.add(8*x -2*y +4*z == 7)
s.add(-4*x + y -2*z == 14)

if s.check() == sat:
    m = s.model()
    print(m)
else:
    print(s.check())

unsat


### Limits of Z3

In [7]:
x= Int("x")

s = Solver()
s.add(2**x == 1024) # 2**10 == 1024
if s.check() == sat:
    m = s.model()
    print(m)
else:
    print(s.check())

unknown


## The Knapsack problem
![title](https://imgs.xkcd.com/comics/np_complete.png)

A variant of the knapsack problem. Normally, taking each item is a binary choice and you aim to maximize value. Here, you can order the same item multiple times and  get the total value exactly. 

[Source](https://www.explainxkcd.com/wiki/index.php/287:_NP-Complete)

In [8]:
a = IntVector('a', 7) # Int vector with 7 variables
s = Solver()

s.add(a[0]* 2.15 + a[1] * 2.75 + a[2] * 3.35 + a[3] * 3.55 + a[4] * 4.20 + a[5] * 5.80  + a[6] * 6.55 == 15.05)
for i in range(7):
    s.add(a[i] >= 0)

s.add(a[0] != 7)
if s.check() == sat:
    m = s.model()
    for i in range(7):
        print(m.evaluate(a[i]),end=",")
    print()
    print(s.model())

1,0,0,2,0,1,0,
[a__0 = 1,
 a__2 = 0,
 a__6 = 0,
 a__5 = 1,
 a__4 = 0,
 a__3 = 2,
 a__1 = 0]


### Optimization using Z3
[The knapsack problem - Wikipedia](https://en.wikipedia.org/wiki/Knapsack_problem)  
Given a *set of items*, each with a *weight* and a *value*, determine which items to include in the collection so that the **total weight is less than or equal to a given limit** and **the total value is as large as possible**.

In [9]:
capacity = 10
weights = [5,2,1,4,1]
values = [4,2,2,10,1]

def solve_knapsack(capacity,weights,values,verbose=False):
    assert len(weights) == len(values)
    o = Optimize() # Create optimize context

    # Create and restrict variables to {0,1}
    variables = [Int(f"x_{i}") for i in range(len(weights))]
    for var in variables:
        o.add(Or(var == 0, var == 1))
    
    weight_product = [weight * var for weight,var in zip(weights,variables)]
    weight = sum(weight_product)
    if verbose:
        print(f"{weight=}")
    value_product = [value * var for value,var in zip(values,variables)]
    value = sum(value_product)

    if verbose:
        print(f"{value=}")
    
    o.add(weight <= capacity)
    o.maximize(value)
    if o.check():
        m = o.model()

        weight = sum([weight * m[var].as_long() for weight,var in zip(weights,variables)])
        value = sum([value * m[var].as_long() for value,var in zip(values,variables)])
        if verbose:
            for var in variables:
                print(m[var],end=",")
            print()
            print(f"{weight=} {value=}")
        return m
    print("unsat")
    return None
    
solve_knapsack(capacity,weights,values,verbose = True)

weight=0 + 5*x_0 + 2*x_1 + 1*x_2 + 4*x_3 + 1*x_4
value=0 + 4*x_0 + 2*x_1 + 2*x_2 + 10*x_3 + 1*x_4
1,0,1,1,0,
weight=10 value=16


[x_0 = 1, x_3 = 1, x_1 = 0, x_2 = 1, x_4 = 0]

In [16]:
import random
import time

for i in range(10,35,2):
    avg = 0
    runs = 10
    for _ in range(runs):
        weights = [random.randrange(1,100) for _ in range(i)]    
        values = [random.randrange(1,100) for _ in range(i)]
        capacity = random.randrange(50,500)
        t0 = time.time()
        solve_knapsack(capacity,weights,values,verbose=False)
        t1 = time.time()
        total = t1-t0
        avg += total

    print(f"{i} : {avg/runs:.2}s.")


10 : 0.0042s.
12 : 0.0045s.
14 : 0.0062s.
16 : 0.0076s.
18 : 0.0095s.
20 : 0.021s.
22 : 0.17s.
24 : 0.12s.
26 : 0.45s.
28 : 0.41s.
30 : 0.84s.
32 : 3.4s.
34 : 3.1s.


### Subset Sum
[Wikipedia](https://en.wikipedia.org/wiki/Subset_sum_problem)

Given a (multi) set of integers $S$ and a target-sum $T$, decide whether any subset of the integers sum to $T$.


In [11]:
S = [-7,-3,-2,9000,5,8]
T = 0
solver = Solver()
variables = [Int(f"x_{i}") for i in range(len(S))]

products = []

for i,s in enumerate(S):
    products.append(variables[i] * S[i])
    solver.add(Or(variables[i]==0,variables[i]==1))

solver.add(sum(products) == T)
solver.add(sum(variables)>=1) # Empty subset not allowed

if solver.check() == sat:
    m = solver.model()
    for i in range(len(S)):
        if m[variables[i]] == 1: # Equivalent to m.evaluate(variables[i])
            print(S[i],end=",")

-3,-2,5,

### Example - Sudoku Solver


x**4 == 16
x + -1*y == 2
x**2 + 2*x*y + y**2
2*x**2 + -1*y**2 == -2


And(A, B)
False


## Proofs
it is enought to find one concrete assignment of variables to say that a statement is **satisfiable**.

A formula is **valid** if it always evaluates to true for **any assignment of variables.**

The Z3 proof function receives a formula as input. It then creates a solver and attempts to solve the **negation** of the input formula. If the exhaustive search finds no valid assignment, then the formula is proven.

### Example 
De Morgan's laws
$$\neg (A \vee B) \equiv (\neg A) \wedge (\neg B) \quad \text{and} \quad \neg (A \wedge B) \equiv (\neg A) \vee (\neg B)$$


In [14]:
# Define variables
A, B = Bools("A B")

de_morgan1 = Not(Or(A,B)) == And(Not(A),Not(B))

de_morgan2 = Not(And(A,B)) == Or(Not(A),Not(B))

prove(de_morgan1)
prove(de_morgan2)

proved
proved


## BitVectors
[Bithacks](https://graphics.stanford.edu/~seander/bithacks.html#IntegerAbs)

Assume you stumble upon the following c code and want to verify it.
```c++
// Returns the absolute value of v
uint32_t branchless_abs(int32_t v) {    
    uint32_t r;  // result
    int32_t const mask = v >> sizeof(int32_t) * CHAR_BIT - 1; // 4*8-1 = 31 for 32 bit integers
    r = (v + mask) ^ mask;
    return r;
}
```

In [15]:
s = Solver()

v = BitVec("v",32)

mask = v >> 31
r = (v + mask) ^ mask

prove(If(v>0,v,-v) == r) 

proved


In Z3Py, the operators <, <=, >, >=, /, % and >> correspond to the **signed versions.** The corresponding unsigned operators are ULT, ULE, UGT, UGE, UDiv, URem and LShR.

## Functions

## References
[Programming Guide - Microsoft](https://z3prover.github.io/papers/programmingz3.html)

[Python Tutorial - Microsoft](https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/Introduction)

[Z3 - Github](https://github.com/Z3Prover/z3)

[SAT-SMT by example](https://smt.st/SAT_SMT_by_example.pdf)

[Z3 Examples by Ericpony](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

[Z3py API](https://z3prover.github.io/api/html/namespacez3py.html)

NameError: name 'Bools' is not defined